In [1]:
import geopandas as gpd

zcta_all = gpd.read_file(r"E:\USC\3\DSCI550\project\zipcode_data\tl_2023_us_zcta520\tl_2023_us_zcta520.shp")
print(zcta_all.columns)


Index(['ZCTA5CE20', 'GEOID20', 'GEOIDFQ20', 'CLASSFP20', 'MTFCC20',
       'FUNCSTAT20', 'ALAND20', 'AWATER20', 'INTPTLAT20', 'INTPTLON20',
       'geometry'],
      dtype='object')


In [2]:
import geopandas as gpd
import pandas as pd


zcta_all = gpd.read_file(r"E:\USC\3\DSCI550\project\zipcode_data\tl_2023_us_zcta520\tl_2023_us_zcta520.shp")

zcta_all["ZIP"] = zcta_all["ZCTA5CE20"].astype(str)

zcta_ca = zcta_all[(zcta_all["ZIP"] >= "90000") & (zcta_all["ZIP"] <= "96199")]

print("California ZIP count:", len(zcta_ca))


California ZIP count: 1802


In [3]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

df = pd.read_csv("sample_40001_80000.csv")


df["geometry"] = df.apply(lambda row: Point(row["longitude"]/1e6, row["latitude"]/1e6), axis=1)
gdf_points = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")


zcta_all = gpd.read_file(r"E:\USC\3\DSCI550\project\zipcode_data\tl_2023_us_zcta520\tl_2023_us_zcta520.shp")
zcta_all["ZIP"] = zcta_all["ZCTA5CE20"].astype(str)
zcta_ca = zcta_all[(zcta_all["ZIP"] >= "90000") & (zcta_all["ZIP"] <= "96199")]
zcta_ca = zcta_ca.to_crs("EPSG:4326")

joined = gpd.sjoin(gdf_points, zcta_ca[["ZIP", "geometry"]], how="left", predicate="within")


joined.drop(columns=["geometry"], inplace=True)   
joined.to_csv("sample_40001_80000_with_zip.csv", index=False)

print(" sample_40001_80000_with_zip.csv")


 sample_40001_80000_with_zip.csv


In [4]:
import geopandas as gpd
import pandas as pd

zcta_all = gpd.read_file(r"E:\USC\3\DSCI550\project\zipcode_data\tl_2023_us_zcta520\tl_2023_us_zcta520.shp")
zcta_all["ZIP"] = zcta_all["ZCTA5CE20"].astype(str)

zcta_ca = zcta_all[(zcta_all["ZIP"] >= "90000") & (zcta_all["ZIP"] <= "96199")]

zcta_ca = zcta_ca.to_crs("EPSG:4326")

df = pd.read_csv("cols_exist.csv")

chunk_size = 50000
output_path = "2016_with_zip.csv"
first = True

for i in range(0, len(df), chunk_size):

    chunk = df.iloc[i:i+chunk_size]
    gdf = gpd.GeoDataFrame(
        chunk,
        geometry=gpd.points_from_xy(chunk.longitude/1e6, chunk.latitude/1e6),
        crs="EPSG:4326"
    )

    joined = gpd.sjoin(gdf, zcta_ca[["ZIP","geometry"]], how="left", predicate="within")
    joined = joined.drop(columns="geometry")

    joined.to_csv(
        output_path,
        mode='w' if first else 'a',
        header=first,
        index=False
    )
    first = False

print("points_with_zip.csv")


points_with_zip.csv
